# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import cv2
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from scipy.misc import imread
from scipy.linalg import norm
from scipy import sum, average

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
# sys.path.append("C:\tensorflow\models\research\object_detection\\")
# sys.path.append("C:\tensorflow\models\research\\")

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model we are using
MODEL_NAME = 'faster_rcnn_resnet101_coco'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = os.path.join('exported-model-02','frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('label_map.pbtxt')

NUM_CLASSES = 1

## Download Model

In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [7]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Static image crater detection

In [8]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test-images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 6) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')        
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image = image.convert("RGB")
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            start = time.time()
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            end = time.time()
            print(end - start)
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=1)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)

# Crater detection in video

In [9]:
vid = 'video1.mp4'
video_path = os.path.join(PATH_TO_TEST_IMAGES_DIR, vid)

In [29]:
snapshots = []

def to_grayscale(arr):
    "If arr is a color image (3D array), convert it to grayscale (2D array)."
    if len(arr.shape) == 3:
        return average(arr, -1)  # average over the last axis (color channels)
    else:
        return arr

def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng

def compare_images(img1, img2):
    # normalize to compensate for exposure difference, this may be unnecessary
    # consider disabling it
    img1 = normalize(img1)
    img2 = normalize(img2)
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = sum(abs(diff))  # Manhattan norm
    z_norm = norm(diff.ravel(), 0)  # Zero norm
    return (m_norm, z_norm)

def least_square_diff(img1, img2):
    dist_euclidean = np.sqrt(np.mean((img1 - img2) ** 2))
    #dist_man = np.mean(abs(img1 - img2))
    #dist_ncc = np.mean( (img1 - np.mean(img1)) * (img2 - np.mean(img2)) ) / (
    #    np.std(img1) * np.std(img2) )
    
    return dist_euclidean

def get_tag(img, coords):
    # Gets tag for the image
    # Loop through existing images in snapshot and compares against each of them
    
    tag = None
    img = Image.fromarray(np.uint8(img)).convert('LA')
    print(coords)
    img = img.crop((int(coords[0]), int(coords[2]), int(coords[1]), int(coords[3])))
    for i, im in enumerate(snapshots):
        # Resize our image
        img = img.resize(im.size, Image.ANTIALIAS)
        # compare image
        im = np.array(im)
        img_temp = np.array(img)
        err = least_square_diff(img_temp, im)
        print("Error " + str(err))
        if err < 7.0:
            tag = i + 1
            break
    
    if tag == None:
        tag = len(snapshots) + 1
        snapshots.append(img)
    
    return tag

def get_bounding_box(image,boxes, scores, max_boxes_to_draw=20, min_score_thresh=.5):
    # Create a display string (and color) for every box location, group any boxes
    # that correspond to the same location.
    box_to_color_map = defaultdict(str)
    indices = []
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        if scores is None or scores[i] > min_score_thresh:
            indices.append(i)
            box = tuple(boxes[i].tolist())
            box_to_color_map[box] = 'black'

    # Draw all boxes onto image.
    results = []
    image = Image.fromarray(np.uint8(image)).convert('RGB')
    im_width, im_height = image.size
    for box, color in box_to_color_map.items():
        ymin, xmin, ymax, xmax = box
        coord = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
        results.append(coord)
    
    return indices, results

In [30]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        
        cap = cv2.VideoCapture(video_path)

        while True:
            ret, image_np = cap.read()
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            #start = time.time()
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            indices, bboxes = get_bounding_box(image_np, np.squeeze(boxes), np.squeeze(scores))
            tags = [0] * np.shape(np.squeeze(scores))[0]
            for ix, box in enumerate(bboxes):
                tag_x = get_tag(image_np, box)
                print(tag_x)
                tags[indices[ix]] = tag_x
            #end = time.time()
            #print(end - start)
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                scores=np.squeeze(scores),
                category_index=category_index,
                use_normalized_coordinates=True,
                line_thickness=1,
                id_tag=tags)
            # Show video
            cv2.imshow('Crater Detector', cv2.resize(image_np, (800,600)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

(627.3212796449661, 680.3237369060516, 391.73208475112915, 454.17587757110596)
1
(622.1901458501816, 681.0744959115982, 382.8376293182373, 450.3037691116333)
Error 6.71035773029
1
(617.3862859010696, 681.7069227695465, 378.91860008239746, 450.42625665664673)
Error 7.03326073126
2
(618.0676975250244, 677.5126737356186, 371.8019127845764, 435.6996774673462)
Error 7.1005907317
Error 6.85535971874
2
(604.3688501119614, 674.0057243108749, 357.07104206085205, 433.3463788032532)
Error 6.96073142498
1
(599.9293092489243, 668.3995854854584, 353.1901717185974, 428.7976026535034)
Error 6.93468436626
1
(591.8980973958969, 661.7064591646194, 349.9956250190735, 424.24163818359375)
Error 6.88961221796
1
(582.9277518987656, 646.3325617313385, 344.51208114624023, 418.09362173080444)
Error 6.65185911678
1
(571.0459612607956, 643.9860769510269, 336.84550523757935, 413.70813846588135)
Error 6.88987303252
1
(568.2178883552551, 638.221984744072, 333.9113116264343, 406.76926374435425)
Error 6.92460358222
1
(

(294.4678524136543, 368.8310409784317, 50.547538697719574, 128.73369455337524)
Error 6.80490107542
1
(187.38251304626465, 227.6367871761322, 133.12325477600098, 177.1644651889801)
Error 7.31646757758
Error 7.29183512833
Error 7.32276867041
Error 7.09912488438
Error 7.45350969869
Error 7.11923016591
Error 5.38167389354
7
(285.3959187269211, 365.01689410209656, 40.863798558712006, 121.23395204544067)
Error 6.99061322457
1
(603.0122828483582, 644.6058691740036, 395.75557708740234, 439.56477642059326)
Error 7.35647333375
Error 7.32666322484
Error 7.07974514053
Error 7.38904144751
Error 7.1586124532
Error 7.2260800165
Error 7.23472665975
Error 7.18089750718
Error 6.79146443842
9
(244.18241411447525, 283.2447187900543, 472.98030853271484, 512.8708362579346)
Error 7.2022196339
Error 7.14683626812
Error 7.2938473828
Error 7.39458208858
Error 7.2792459351
Error 7.08091893739
Error 6.98670281123
7
(285.3706085681915, 359.5965539216995, 40.83697199821472, 114.81727659702301)
Error 6.96594634673
1

(571.5315155982971, 624.2270616292953, 90.94575047492981, 148.1878101825714)
Error 7.38755175511
Error 7.45322842507
Error 6.82145111161
3
(528.302377820015, 569.7088161706924, 325.10268688201904, 368.2472348213196)
Error 7.45032465911
Error 7.33854874576
Error 7.02950185832
Error 7.17002470482
Error 7.27091997814
Error 7.21390541017
Error 7.32694935715
Error 7.30709796997
Error 6.78067387397
9
(206.13065534830093, 249.35670602321625, 488.2852077484131, 528.5843253135681)
Error 7.1737516954
Error 7.19882981073
Error 7.30477162747
Error 7.36478390238
Error 7.22679492166
Error 7.32665300562
Error 7.11972444625
Error 7.1946267095
Error 7.39660688095
Error 6.80439492948
10
(608.2817842960358, 644.1291877031326, 259.13334488868713, 298.8306999206543)
Error 7.31318190187
Error 7.32013023645
Error 6.94904010858
3
(567.8067924976349, 616.6574432849884, 80.32589256763458, 144.38504576683044)
Error 7.42963396921
Error 7.43456027167
Error 6.99976468179
3
(602.0082178115845, 644.3701862096786, 249

(506.20113015174866, 550.3230745792389, 25.247177481651306, 70.91947048902512)
Error 7.45189223041
Error 7.12901473929
Error 6.9379010802
3
(145.00440379977226, 182.5108186006546, 429.63019609451294, 465.9497022628784)
Error 7.15008319724
Error 7.29647479776
Error 7.34267966033
Error 7.34167009585
Error 7.30601175244
Error 7.22435980991
Error 7.17668390372
Error 7.43151823737
Error 7.44055004456
Error 7.09527427525
Error 6.76684613751
11
(466.0002158880234, 507.7548631429672, 252.41024494171143, 297.4067151546478)
Error 7.52273184817
Error 7.47533630615
Error 7.02278838878
Error 7.07901538493
Error 7.29304665417
Error 7.26621247045
Error 7.35548602087
Error 7.34346478119
Error 6.46470825561
9
(459.69933569431305, 502.5207713842392, 246.39639258384705, 295.0759828090668)
Error 7.46922406787
Error 7.45733597461
Error 7.03496380823
Error 7.03484673489
Error 7.31914825079
Error 7.2556016715
Error 7.27352479359
Error 7.22939492236
Error 6.82462245828
9
(140.92123937606812, 176.5861561298370

(466.814270734787, 504.91701781749725, 113.45531344413757, 152.43651866912842)
Error 7.37552186975
Error 7.47219063088
Error 6.89018817028
3
(68.10844114422798, 107.69863027334213, 343.9436674118042, 383.36126804351807)
Error 7.20665726636
Error 7.18833828139
Error 7.3506493266
Error 7.33579351039
Error 7.16904304612
Error 7.19380452889
Error 7.24746550281
Error 7.38821049968
Error 7.34985478624
Error 7.1696174372
Error 7.13327749346
Error 7.18478559563
Error 7.41249238962
Error 7.25354784049
Error 6.872332993
15
(386.88928723335266, 432.14826703071594, 180.61603903770447, 223.38114380836487)
Error 7.49471212211
Error 7.54252196656
Error 7.14423766234
Error 7.11593759351
Error 7.3917768351
Error 7.30997669007
Error 7.31876974198
Error 7.14412237971
Error 6.97254273571
9
(14.121963296085596, 59.40204298496246, 231.04255199432373, 278.7592828273773)
Error 7.32895197058
Error 7.35677866142
Error 7.2558286925
Error 6.84242705671
4
(21.794926770031452, 79.55036555230618, 140.61817824840546,

(407.3764091730118, 447.0083968639374, 49.452562630176544, 94.45432126522064)
Error 7.37094211488
Error 7.37391775604
Error 7.17993819116
Error 7.13471533581
Error 7.27473936881
Error 7.16534495428
Error 7.26971506408
Error 7.1743361457
Error 7.0481487016
Error 7.36041108856
Error 7.25065704286
Error 7.35825423323
Error 7.10236199949
Error 7.364743237
Error 7.20160625656
Error 7.19017123884
Error 7.32397507924
Error 7.31797174033
Error 7.1966661444
Error 7.15608090091
21
(6.186436953023076, 48.32392655313015, 282.2697043418884, 329.4760823249817)
Error 7.09152715657
Error 7.19846577777
Error 7.16413273493
Error 7.40751098061
Error 7.2853736803
Error 7.2996141389
Error 7.32595806535
Error 7.35327680999
Error 7.32529372259
Error 7.08717590399
Error 7.05762869463
Error 7.15359031867
Error 7.28331796471
Error 7.17829037101
Error 6.18885181536
15
(401.7702294588089, 443.0423072576523, 45.00887542963028, 89.48944509029388)
Error 7.43272714179
Error 7.43112530109
Error 7.16580470798
Error 6.9